In [1]:
import os

img_test_dir = "img_test"

In [2]:
# check if dir is there
if os.path.exists(img_test_dir):
    print("Path exists")
else:
    print("nope")

Path exists


In [3]:
label_dict = {
    "rotten banana": 0,
    "rotten apple": 1,
    "fresh banana": 2,
    "fresh mango": 3,
    "fresh orange": 4,
    "fresh apple": 5,
    "rotten mango": 6,
    "rotten orange": 7
}

label_dict_no_ws = {
    "rotten_banana": 0,
    "rotten_apple": 1,
    "fresh_banana": 2,
    "fresh_mango": 3,
    "fresh_orange": 4,
    "fresh_apple": 5,
    "rotten_mango": 6,
    "rotten_orange": 7
}

In [4]:
labels = list(label_dict.keys())
labels

['rotten banana',
 'fresh orange',
 'rotten mango',
 'rotten apple',
 'fresh mango',
 'fresh apple',
 'fresh banana',
 'rotten orange']

In [5]:
# eliminate the whitespace in labels
labels_no_whitespace = []
for label in labels:
    labels_no_whitespace.append(label.replace(" ", "_"))

In [6]:
labels_no_whitespace

['rotten_banana',
 'fresh_orange',
 'rotten_mango',
 'rotten_apple',
 'fresh_mango',
 'fresh_apple',
 'fresh_banana',
 'rotten_orange']

In [7]:
def get_ground_truth(path):
    images = os.listdir(path)
    print("Total {} images in {}".format(len(images), path))
    
    ground_truth_dict = {}
    
    for img in images:
        for label in labels_no_whitespace:
            if label in img:
                img_path = os.path.join(img_test_dir, img)
                ground_truth_dict[img_path] = label_dict_no_ws[label]
                
    return ground_truth_dict

In [8]:
ground_truth = get_ground_truth(img_test_dir)

Total 552 images in img_test


In [9]:
# pickle the ground_truths
import pickle

pickle_dir = "inception_pickled"
pickle_out_file = os.path.join(pickle_dir, "ground_truth.pickle")

gt_pickle_out = open(pickle_out_file, "wb")
pickle.dump(ground_truth, gt_pickle_out)

In [10]:
# check if pickled properly
pickle_in = pickle.load(open(pickle_out_file, "rb"))
print(pickle_in)

{'img_test/rotten_apple_17.jpeg': 1, 'img_test/rotten_orange_44.jpeg': 7, 'img_test/fresh_apple_68.jpeg': 5, 'img_test/rotten_orange_73.jpg': 7, 'img_test/fresh_apple_120.jpeg': 5, 'img_test/fresh_apple_108.jpeg': 5, 'img_test/fresh_apple_31.jpeg': 5, 'img_test/rotten_orange_85.jpeg': 7, 'img_test/fresh_apple_21.jpeg': 5, 'img_test/fresh_apple_313.jpeg': 5, 'img_test/rotten_orange_75.jpg': 7, 'img_test/fresh_apple_48.jpeg': 5, 'img_test/rotten_apple_16.jfif': 1, 'img_test/rotten_orange_78.jpeg': 7, 'img_test/fresh_apple_224.jpeg': 5, 'img_test/fresh_orange_4.jpeg': 4, 'img_test/fresh_apple_187.jpeg': 5, 'img_test/fresh_apple_142.jpeg': 5, 'img_test/fresh_apple_272.jpeg': 5, 'img_test/fresh_apple_138.jpeg': 5, 'img_test/fresh_apple_64.jpeg': 5, 'img_test/fresh_apple_132.jpeg': 5, 'img_test/rotten_orange_63.jpeg': 7, 'img_test/rotten_orange_62.jpeg': 7, 'img_test/fresh_orange_76.jpeg': 4, 'img_test/fresh_orange_71.jpg': 4, 'img_test/fresh_apple_302.jfif': 5, 'img_test/fresh_apple_292.jpe

In [11]:
# now for classification

import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def classify(image_path):
    # Read in the image_data
    image_data = tf.gfile.FastGFile(image_path, 'rb').read()

    # Loads label file, strips off carriage return
    label_lines = [line.rstrip() for line
                   in tf.gfile.GFile("retrained_labels.txt")]

    # label_names = label_lines

    # Unpersists graph from file
    with tf.gfile.FastGFile("retrained_graph.pb", 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name='')

    with tf.Session() as sess:
        # Feed the image_data as input to the graph and get first prediction
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')

        predictions = sess.run(softmax_tensor, \
                               {'DecodeJpeg/contents:0': image_data})

        # global_pred = predictions

        # Sort to show labels of first prediction in order of confidence
        top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]
        # label_id = top_k

        
        print('Processing {} ...................DONE'.format(image_path))
        resultDict = {}
        for node_id in top_k:
            human_string = label_lines[node_id]
            score = predictions[0][node_id]
#              print('Category : {} \nScore = {} %'.format(human_string, score * 100))

            resultDict[human_string] = score
        
        #print('Category : {}\tScore : {}'.format(label_lines[node_id], predictions[0][node_id]))

        #print('\nDONE===========================', end='\n')
        
        return resultDict


In [12]:
image_dir = img_test_dir

images = os.listdir(image_dir)
label_lines = [line.rstrip() for line
                   in tf.gfile.GFile("retrained_labels.txt")]
pred_dict = {}
    
# create graph
# with tf.gfile.FastGFile("retrained_graph.pb", 'rb') as f:
#     graph_def = tf.GraphDef()
#     graph_def.ParseFromString(f.read())
#     tf.import_graph_def(graph_def, name='')
    
def create_graph():
  """Creates a graph from saved GraphDef file and returns a saver."""
  # Creates graph from saved graph_def.pb.
  with tf.gfile.FastGFile("retrained_graph.pb", 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')


In [13]:
# def run_inference(img_dir):
#     create_graph()
    
#     pred_dict = {}
    
#     for image in images:
#         image = os.path.join(image_dir, image)
#         image_data = tf.gfile.FastGFile(image, 'rb').read()
        
#         with tf.Session() as sess:
#             # Feed the image_data as input to the graph and get first prediction
#             softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')

#             predictions = sess.run(softmax_tensor, \
#                                     {'DecodeJpeg/contents:0': image_data})


#             # Sort to show labels of first prediction in order of confidence
#             top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]
#             # label_id = top_k

        
#             print('Processing {} ...................DONE'.format(image))
#             resultDict = {}
#             for node_id in top_k:
#                 human_string = label_lines[node_id]
#                 score = predictions[0][node_id]
                
#                 resultDict[human_string] = score
                
#         # get max score
#         maxKey = max(rDict.keys(), key=(lambda k: rDict[k]))
#         pred_dict[image] = label_dict_no_ws[maxKey]
        
#         return pred_dict

In [14]:
def run_inference(img_dir):
    create_graph()
    
    pred_dict = {}
        
    with tf.Session() as sess:
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
        for image in images:
            image = os.path.join(image_dir, image)
            image_data = tf.gfile.FastGFile(image, 'rb').read()
                # Feed the image_data as input to the graph and get first prediction
            

            predictions = sess.run(softmax_tensor, \
                                        {'DecodeJpeg/contents:0': image_data})


                # Sort to show labels of first prediction in order of confidence
            top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]
                # label_id = top_k

        
            print('Processing {} ...................DONE'.format(image))
            resultDict = {}
            for node_id in top_k:
                human_string = label_lines[node_id]
                score = predictions[0][node_id]
                
                resultDict[human_string] = score
                
            # get max score
            maxKey = max(resultDict.keys(), key=(lambda k: resultDict[k]))
            pred_dict[image] = label_dict[maxKey]
        
    return pred_dict

In [15]:
d = run_inference(img_dir=img_test_dir)

Processing img_test/fresh_apple_4.jpeg ...................DONE
Processing img_test/fresh_apple_1.jpeg ...................DONE
Processing img_test/fresh_apple_10.jpeg ...................DONE
Processing img_test/fresh_apple_100.jpeg ...................DONE
Processing img_test/fresh_apple_101.jpeg ...................DONE
Processing img_test/fresh_apple_102.jpeg ...................DONE
Processing img_test/fresh_apple_103.jpeg ...................DONE
Processing img_test/fresh_apple_104.jpeg ...................DONE
Processing img_test/fresh_apple_105.jpeg ...................DONE
Processing img_test/fresh_apple_106.jpeg ...................DONE
Processing img_test/fresh_apple_107.jpeg ...................DONE
Processing img_test/fresh_apple_108.jpeg ...................DONE
Processing img_test/fresh_apple_109.jpeg ...................DONE
Processing img_test/fresh_apple_11.jpeg ...................DONE
Processing img_test/fresh_apple_110.jpeg ...................DONE
Processing img_test/fresh_apple

Processing img_test/fresh_apple_150.jpeg ...................DONE
Processing img_test/fresh_apple_151.jpeg ...................DONE
Processing img_test/fresh_apple_152.jpeg ...................DONE
Processing img_test/fresh_apple_153.jpeg ...................DONE
Processing img_test/fresh_apple_154.jpeg ...................DONE
Processing img_test/fresh_apple_155.jpeg ...................DONE
Processing img_test/fresh_apple_156.jpeg ...................DONE
Processing img_test/fresh_apple_157.jpeg ...................DONE
Processing img_test/fresh_apple_158.jpeg ...................DONE
Processing img_test/fresh_apple_159.jpeg ...................DONE
Processing img_test/fresh_apple_160.jpeg ...................DONE
Processing img_test/fresh_apple_161.jpeg ...................DONE
Processing img_test/fresh_apple_162.jpeg ...................DONE
Processing img_test/fresh_apple_163.jpeg ...................DONE
Processing img_test/fresh_apple_164.jpeg ...................DONE
Processing img_test/fresh

Processing img_test/fresh_apple_29.jpg ...................DONE
Processing img_test/fresh_apple_290.jpeg ...................DONE
Processing img_test/fresh_apple_291.jpeg ...................DONE
Processing img_test/fresh_apple_292.jpeg ...................DONE
Processing img_test/fresh_apple_293.jpeg ...................DONE
Processing img_test/fresh_apple_294.jpeg ...................DONE
Processing img_test/fresh_apple_295.jpeg ...................DONE
Processing img_test/fresh_apple_296.jpeg ...................DONE
Processing img_test/fresh_apple_297.jpeg ...................DONE
Processing img_test/fresh_apple_298.jpeg ...................DONE
Processing img_test/fresh_apple_299.jpeg ...................DONE
Processing img_test/fresh_apple_3.jpeg ...................DONE
Processing img_test/fresh_apple_30.jpeg ...................DONE
Processing img_test/fresh_apple_301.jfif ...................DONE
Processing img_test/fresh_apple_302.jfif ...................DONE
Processing img_test/fresh_appl

Processing img_test/fresh_orange_7.jpeg ...................DONE
Processing img_test/fresh_orange_70.jpg ...................DONE
Processing img_test/fresh_orange_71.jpg ...................DONE
Processing img_test/fresh_orange_73.jpeg ...................DONE
Processing img_test/fresh_orange_74.jpeg ...................DONE
Processing img_test/fresh_orange_75.jpeg ...................DONE
Processing img_test/fresh_orange_76.jpeg ...................DONE
Processing img_test/fresh_orange_77.jpeg ...................DONE
Processing img_test/fresh_orange_78.jpeg ...................DONE
Processing img_test/fresh_orange_79.jpg ...................DONE
Processing img_test/fresh_orange_8.jpeg ...................DONE
Processing img_test/fresh_orange_9.jpeg ...................DONE
Processing img_test/fresh_orange_a.jpeg ...................DONE
Processing img_test/fresh_orange_b.jpg ...................DONE
Processing img_test/rotten_apple_1.jfif ...................DONE
Processing img_test/rotten_apple_10

Processing img_test/rotten_orange_87.jpeg ...................DONE
Processing img_test/rotten_orange_88.jpeg ...................DONE
Processing img_test/rotten_orange_89.jpeg ...................DONE
Processing img_test/rotten_orange_9.jpg ...................DONE
Processing img_test/rotten_orange_90.jpeg ...................DONE
Processing img_test/rotten_orange_91.jpeg ...................DONE
Processing img_test/rotten_orange_92.jpeg ...................DONE
Processing img_test/rotten_orange_93.jpeg ...................DONE
Processing img_test/rotten_orange_94.jpeg ...................DONE
Processing img_test/rotten_orange_95.jpeg ...................DONE
Processing img_test/rotten_orange_96.jpeg ...................DONE
Processing img_test/rotten_orange_97.jpeg ...................DONE
Processing img_test/rotten_orange_98.jpeg ...................DONE
Processing img_test/rotten_orange_99.jpeg ...................DONE
Processing img_test/rotten_orange_a.jpeg ...................DONE
Processing im

In [16]:
result_dict = d

In [17]:
len(result_dict)

552

In [18]:
# pickle again

result_pickle_file = os.path.join(pickle_dir, "inception_predition.pickle")
pickle_out = open(result_pickle_file, "wb")
pickle.dump(result_dict, pickle_out)

pickle_out.close()

In [19]:
# check again
pickle_load = pickle.load(open(result_pickle_file, "rb"))
print(pickle_load)

{'img_test/rotten_apple_17.jpeg': 1, 'img_test/rotten_orange_44.jpeg': 4, 'img_test/fresh_apple_68.jpeg': 5, 'img_test/rotten_orange_73.jpg': 3, 'img_test/fresh_apple_120.jpeg': 5, 'img_test/fresh_apple_108.jpeg': 5, 'img_test/fresh_apple_31.jpeg': 1, 'img_test/rotten_orange_85.jpeg': 1, 'img_test/fresh_apple_21.jpeg': 5, 'img_test/fresh_apple_313.jpeg': 5, 'img_test/rotten_orange_75.jpg': 0, 'img_test/fresh_apple_48.jpeg': 5, 'img_test/rotten_apple_16.jfif': 1, 'img_test/rotten_orange_78.jpeg': 7, 'img_test/fresh_apple_224.jpeg': 5, 'img_test/fresh_orange_4.jpeg': 4, 'img_test/fresh_apple_187.jpeg': 5, 'img_test/fresh_apple_142.jpeg': 5, 'img_test/fresh_apple_272.jpeg': 5, 'img_test/fresh_apple_138.jpeg': 5, 'img_test/fresh_apple_64.jpeg': 5, 'img_test/fresh_apple_132.jpeg': 5, 'img_test/rotten_orange_63.jpeg': 7, 'img_test/rotten_orange_62.jpeg': 6, 'img_test/fresh_orange_76.jpeg': 4, 'img_test/fresh_orange_71.jpg': 4, 'img_test/fresh_apple_302.jfif': 5, 'img_test/fresh_apple_292.jpe